In [1]:
import torch

  创建一个5*3的未初始化的tensor

In [2]:
x = torch.empty(5,3)
print(x)

tensor([[9.2755e-39, 9.1837e-39, 9.3674e-39],
        [1.0745e-38, 1.0653e-38, 9.5510e-39],
        [1.0561e-38, 1.0194e-38, 1.1112e-38],
        [1.0561e-38, 9.9184e-39, 1.0653e-38],
        [4.1327e-39, 1.0194e-38, 1.0469e-38]])


  创建一个5*3的long型全0的tensor

In [4]:
x = torch.zeros(5, 3, dtype=torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])


可以使用shape属性或size方法来获取tensor的形状,返回一个元组

In [6]:
# x.size()
print(x.shape)

torch.Size([5, 3])


## 2.操作 
### 1. 加法

In [12]:
y = torch.rand(5,3)
result = torch.empty(5, 3)
# print(x+y)
# print(torch.add(x, y, out=result))
y.add_(x)
print(y)

tensor([[0.1371, 0.6264, 0.8099],
        [0.2090, 0.6607, 0.5775],
        [0.9998, 0.4596, 0.2482],
        [0.6568, 0.3034, 0.9711],
        [0.3780, 0.7060, 0.0399]])


### 2.索引
索引出来的数据与原数据共享内存

In [13]:
y = x[0, :]
y += 1
print(y, '\n', x[0])

tensor([1, 1, 1]) 
 tensor([1, 1, 1])


改变形状： 使用view来改变形状—— 同样的，view()也是与原数据共享内存，仅仅是改变观察角度

In [21]:
y = x.view(15)
z = x.view(-1,5)
print(x.shape, y.shape, z.shape)

torch.Size([5, 3]) torch.Size([15]) torch.Size([3, 5])


  如果想返回一个真正的副本，应该先使用.clone()来返回一个副本，在使用.view()来改变形状  
 使用clone的另一个好处是**结果会保存在计算图中，即梯度回传到副本时也会传到源tensor**  
 不推荐使用reshape,因为此函数并不保证返回的是其拷贝

In [23]:
x_cp = x.clone().view(15)
x -= 1
print(x_cp, '\n',  x)

tensor([ 0,  0,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]) 
 tensor([[-1, -1, -1],
        [-2, -2, -2],
        [-2, -2, -2],
        [-2, -2, -2],
        [-2, -2, -2]])


另一个常用的函数：item: 将一个标量tensor转换成python num

In [26]:
z = torch.randn(1)
print(z, z.item())

tensor([-1.5209]) -1.5209071636199951


### 3. 广播机制
  

In [32]:
x = torch.arange(1,3).view(1,2)
print(x)
y = torch.arange(1,4).view(3,1)
print(y)
print(x+y)

tensor([[1, 2]])
tensor([[1],
        [2],
        [3]])
tensor([[2, 3],
        [3, 4],
        [4, 5]])


### 4. 运算的内存开销
  索引操作不会开辟新的空间，而像*y = x + y*这样的操作，会开辟新的内存，然后将y指向新的内存。
使用python自带的id函数来验证这一点

In [33]:
x = torch.tensor([1,2])
y = torch.tensor([3,4])
id_before = id(y)
y = y+x
print(id(y) == id_before)

False


想在原内存中保存结果，可以使用y[:] = ...来做到这一点；或者使用全名运算符中的out参数来做到这一点

### 5. tensor和numpy互换
torch.numpy()将tensor转换为nuumpy数组
torch.from_numpy()将numpy转换为tensor;——这两种方式共享内存  
也可以使用torch.tensor()将numpy转换为tensor，但是返回的tensor与元数据不共享内存

In [36]:
a = torch.tensor([1,2])
b = a.numpy()
c = torch.from_numpy(b)
print(type(a),type(b),type(c))

<class 'torch.Tensor'> <class 'numpy.ndarray'> <class 'torch.Tensor'>


### 6. tensor on GPU
使用方法 .to() 可将tensor在CPU和GPU之间相互移动

In [39]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # GPU
    y = torch.ones_like(x, device=device)  # 直接创建在GPU上的tensor
    x = x.to(device)  # 等价于 .to("cuda")
    z = x+y
    print(z)
    print(z.to("cpu", torch.double))

tensor([2, 3], device='cuda:0')
tensor([2., 3.], dtype=torch.float64)
